# Advanced Modeling of Groundwater Flow (GW3099)
![header](../img/header.jpg)

# Exercise 7: MODFLOW 6

The purpose of this exercise is to load and existing model with SFR and LAK packages and add a MVR package to transfer water between the SFR and LAK packages. We will also convert the SFR inflows to use a time series file and add LAK and SFR observations.

In [ ]:
import os
import numpy as np
import flopy
import matplotlib.pyplot as plt

In [ ]:
# load the existing model
model_ws = './ex07-data'
s = flopy.mf6.MFSimulation().load(sim_ws=model_ws)

In [ ]:
# change work space
model_ws = './ex07-completed'
s.set_sim_path(model_ws)

In [ ]:
# get the groundwater model
gwf = s.get_model('ex07')

### Create the MVR package

In [ ]:
# connect sfr reach 6 to lake 1 and
# connect lake 1 to sfr reach 7
mvrperioddata = [['SFR-1', 5, 'LAK-1', 0, 'FACTOR',  1.],
                 ['LAK-1', 0, 'SFR-1', 6, 'FACTOR',  1.]]

In [ ]:
mvr = flopy.mf6.ModflowGwfmvr(gwf, maxmvr=2, maxpackages=2, packages=['SFR-1', 'LAK-1'], perioddata=mvrperioddata)

### Turn on the mover in the SFR and LAK packages

In [ ]:
gwf.lak.mover = True
gwf.sfr.mover = True

### Get a list of the available packages

In [ ]:
gwf.package_dict.keys()

### Add observations to the SFR and LAK packages

In [ ]:
# add sfr observations
sfr = gwf.get_package('sfr')
sfrobsname = 'ex07.sfr.obs'
sfr.obs_filerecord.set_data([sfrobsname])
sfr_obs = [('SFR06-S', 'STAGE', 5), ('SFR06-Q', 'DOWNSTREAM-FLOW', 5),
           ('SFR07-S', 'STAGE', 6), ('SFR07-Q', 'DOWNSTREAM-FLOW', 6),
           ('SFR38-S', 'STAGE', 37), ('SFR38-Q', 'DOWNSTREAM-FLOW', 37),
           ('SEG01', 'SFR', 'SEG1'), ('SEG02', 'SFR', 'SEG2'),
           ('SEG03', 'SFR', 'SEG3'), ('SEG04', 'SFR', 'SEG4')]
sfrobs = flopy.mf6.ModflowUtlobs(gwf, continuous={'ex07.sfr.obs.csv': sfr_obs}, parent_file=sfr, fname=sfrobsname)

In [ ]:
lak = gwf.get_package('lak')
lakobsname = 'ex07.lak.obs'
lak.obs_filerecord.set_data([lakobsname])
lak_obs = [('LAK1-S', 'STAGE', 1),
           ('LAK2-S', 'STAGE', 2),
           ('LAK1-Q', 'LAK', 'LAKE1'), 
           ('LAK2-Q', 'LAK', 'LAKE2')]
lakobs = flopy.mf6.ModflowUtlobs(gwf, continuous={'ex07.lak.obs.csv': lak_obs}, parent_file=lak, fname=lakobsname)

### Change the SFR package to use a time series for inflows

### Write the MODFLOW 6 files and run the model

In [ ]:
s.write_simulation()
s.run_simulation()

In [ ]:
# load sfr and lak obs
sfrobs = np.genfromtxt(os.path.join(model_ws, 'ex07.sfr.obs.csv'), delimiter=',', names=True)
lakobs = np.genfromtxt(os.path.join(model_ws, 'ex07.lak.obs.csv'), delimiter=',', names=True)
lakobs.dtype

In [ ]:
# create mapping array for lake data
lakeconn = gwf.lak.connectiondata.get_data()
lakmap = {0: [], 1: []}
for v in lakeconn:
    if v['claktype'].upper() == 'VERTICAL':
        cid = v['cellid']
        lakmap[v['lakeno']].append((0, cid[1], cid[2]))

In [ ]:
def sub_lake(h, ls, tmap):
    for key, value in tmap.items():
        s = ls[key]
        for loc in value:
            h[loc] = s
    return h    

In [ ]:
# retrieve the heads
hobj = flopy.utils.HeadFile(os.path.join(model_ws, 'ex07.hds'))
times= hobj.get_times()

In [ ]:
# create a spatial reference from the grb
grb = flopy.utils.MfGrdFile(os.path.join(model_ws, 'ex07.dis.grb'), )
sr = grb.get_spatialreference()
sr.get_extent()

In [ ]:
h = hobj.get_data(totim=times[0])
ls = [lakobs['LAK1S'][0], lakobs['LAK2S'][0]]
h = sub_lake(h, ls, lakmap)
mm = flopy.plot.ModelMap(sr=sr)
mm.plot_array(h, masked_values=[1e+30])
c = mm.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d');

In [ ]:
h = hobj.get_data(totim=times[-1])
ls = [lakobs['LAK1S'][-1], lakobs['LAK2S'][-1]]
h = sub_lake(h, ls, lakmap)
mm = flopy.plot.ModelMap(sr=sr)
mm.plot_array(h, masked_values=[1e+30])
c = mm.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d');

In [ ]:
# plot the sfr stage results
for name in sfrobs.dtype.names[1:]:
    if name[-1] == 'S':
        plt.semilogx(sfrobs['time'], sfrobs[name], marker='.', label=name)
plt.legend();

In [ ]:
# plot the sfr Q results
for name in sfrobs.dtype.names[1:]:
    if name[-1] == 'Q':
        plt.semilogx(sfrobs['time'], sfrobs[name], marker='.', label=name)
plt.legend();

In [ ]:
# plot the lake results
for name in lakobs.dtype.names[1:]:
    if name[-1] == 'S':
        plt.semilogx(lakobs['time'], lakobs[name], marker='.', label=name)
plt.legend();